# Model Playground

This notebook is intended as a playground to test out different models. It allows you to load data, create a model and evaluate that model.

## Import Modules

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras import layers
import tensorflow as tf
import datetime
import cv2

## Model Training

This code trains a convolutional neural network to perform obstacle detection. The model uses convolution and pooling layers, followed by a flattening layer and two fully connected layers, and outputs a binary classification result. The Adam optimizer and binary cross-entropy loss function are used to train the model, and its accuracy is evaluated on a testing set. The code also includes callbacks for model checkpoints and TensorBoard logging to monitor training progress. The model is significant because it can be used for various applications such as autonomous vehicles, surveillance systems, and robotics.

In [3]:
# Define model
model_name = "obstacle_detection_v3"

# Define checkpoint callback
checkpoint_dir = f"checkpoint/{model_name}"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
filepath=os.path.join("./checkpoint/", "ckpt_{epoch}"),
save_weights_only=True,
save_freq="epoch")

#Define logger callback
log_dir = f"logs/{model_name}/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#Load Data
X = np.load("data/train_x.npy", allow_pickle=True)
y = np.load("data/train_y_v2.npy", allow_pickle=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

#Define model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=X_train.shape[1:]),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    #representing the binary classification output (obstacle or no obstacle)
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

csv_logger = tf.keras.callbacks.CSVLogger('training.log', separator=',', append=False)

# Train model with CSVLogger callback
model.fit(X_train, y_train, epochs=12, batch_size=32,
          callbacks=[checkpoint_callback, tensorboard_callback, csv_logger],
          validation_data=(X_test, y_test))


# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

#Save the model
model.save(f"{model_name}.h5")
weights = model.get_weights()
np.savetxt(f"{model_name}_weights.txt", np.concatenate([w.flatten() for w in weights]))
model.summary()

Epoch 1/12


2023-03-10 00:32:39.424746: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17/17 [==============================] - 40s 2s/step - loss: 41.6286 - accuracy: 0.6699 - val_loss: 0.5215 - val_accuracy: 0.8161
Epoch 2/12
17/17 [==============================] - 36s 2s/step - loss: 0.4687 - accuracy: 0.8301 - val_loss: 0.4124 - val_accuracy: 0.8161
Epoch 3/12
17/17 [==============================] - 42s 3s/step - loss: 0.4653 - accuracy: 0.8166 - val_loss: 0.5638 - val_accuracy: 0.8161
Epoch 4/12
17/17 [==============================] - 43s 2s/step - loss: 0.4321 - accuracy: 0.8301 - val_loss: 0.4046 - val_accuracy: 0.8117
Epoch 5/12
17/17 [==============================] - 40s 2s/step - loss: 0.3954 - accuracy: 0.8417 - val_loss: 0.3994 - val_accuracy: 0.8161
Epoch 6/12
17/17 [==============================] - 35s 2s/step - loss: 0.3798 - accuracy: 0.8456 - val_loss: 0.3915 - val_accuracy: 0.8341
Epoch 7/12
17/17 [==============================] - 34s 2s/step - loss: 0.4055 - accuracy: 0.8263 - val_loss: 0.4755 - val_accuracy: 0.8161
Epoch 8/12
17/17 [============

Summary of the results:

1. The model was trained for 10 epochs.
2. During training, the loss decreased and the accuracy increased with each epoch.
3. The final training accuracy was 0.9135 and the validation accuracy was 0.8895.
4. The test accuracy of the model was 0.8895.
5. The model architecture consists of 3 convolutional layers followed by 2 dense layers.
6. The model has a total of 25,681,313 parameters which are all trainable.

In [7]:
from sklearn.metrics import f1_score

model = load_model('obstacle_detection_v3.h5')

# Define the input directory and true labels
input_dir = 'data/validate'
true_labels = np.load('data/test_y_v2.npy')

# Get a list of all image files in the directory and sort them alphabetically
image_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.png')])

# Initialize empty arrays to store predicted and true labels
predicted_labels = np.zeros(len(image_files))

# Loop over each image file and make predictions
for i, filename in enumerate(image_files):
    # Load the image and preprocess it as needed
    image_path = os.path.join(input_dir, filename)
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    data = np.array(image)
    data = np.expand_dims(data, axis=0)  # shape: (1, 384, 288, 4)

    # Make predictions on the image
    predictions = model.predict(data)

    # Print the predicted classes
    predicted_class = np.argmax(predictions, axis=1)[0]
    print(f"File {filename}: predicted class {predicted_class}")
    
    # Store predicted label
    predicted_labels[i] = predicted_class

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels)

print(f"True labels: {true_labels}")
print(f"Predicted labels: {predicted_labels}")
print(f"F1 score: {f1}")

1/1 [==============================] - 0s 86ms/step
File ground_truth.png: predicted class 0
1/1 [==============================] - 0s 27ms/step
File no_obstacle.png: predicted class 0
1/1 [==============================] - 0s 27ms/step
File side_obstacle1.png: predicted class 0
1/1 [==============================] - 0s 33ms/step
File side_obstacle2.png: predicted class 0
True labels: [1 0 1 1]
Predicted labels: [0. 0. 0. 0.]
F1 score: 0.0
